# Рекуррентные нейронные сети
 Сильченко Алексей Евгеньевич

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.backend import exp, mean

In [2]:
# Подготовка данных
text = open('text.txt', encoding='utf-8').read().lower()
tokenizer = Tokenizer(char_level=True) #Таким образом, каждый уникальный символ в тексте будет рассматриваться как отдельный токен.
tokenizer.fit_on_texts([text])
encoded = tokenizer.texts_to_sequences([text])[0] # Это означает, что токенизатор анализирует текст, определяет все уникальные символы и создает для них внутренний словарь (индекс)
vocab_size = len(tokenizer.word_index) + 1 #Вычисляет размер словаря

In [3]:
# Создание последовательностей
sequence_length = 10 #длинна последовательности
sequences = [] #список для хранения последовательностей
for i in range(sequence_length, len(encoded)): #проход по закодированному тексту
    sequences.append(encoded[i - sequence_length:i + 1]) #добавление среза закодированного текста
sequences = np.array(sequences) #последовательностей преобразуется в массив NumPy

In [4]:
# Разделение на входные и выходные данные
#разделяет массив последовательностей на входные данные для модели (X) и целевую переменную (y), 
#затем преобразует их в формат, подходящий для обучения модели глубокого обучения.
X, y = sequences[:,:-1], sequences[:,-1]
X = pad_sequences(X, maxlen=sequence_length)
y = to_categorical(y, num_classes=vocab_size)

In [5]:
# Создание модели
model = Sequential() #Инициализация последовательной модели.
model.add(Embedding(vocab_size, 50, input_length=sequence_length)) #слой преобразует входные данные, представленные индексами слов, в плотные векторы фиксированного размера
model.add(SimpleRNN(100, return_sequences=True))  # добавление слоя со 100 нейронами указывает, что слой должен возвращать последовательность выходов для каждого временного шага
#model.add(Dropout(0.2))  
model.add(SimpleRNN(100)) #Данный слой обеспечивает сводку информации из предыдущих временных шагов.
#model.add(Dropout(0.2)) # Добавление Dropout
model.add(Dense(vocab_size, activation='softmax')) #Добавление полносвязного слоя (Dense), преобразует вектор в распределение вероятностей
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
# Обучение модели
model.fit(X, y, epochs=60, verbose=2)

Epoch 1/60
329/329 - 4s - loss: 3.2666 - accuracy: 0.1378 - 4s/epoch - 11ms/step
Epoch 2/60
329/329 - 1s - loss: 2.7710 - accuracy: 0.2413 - 1s/epoch - 3ms/step
Epoch 3/60
329/329 - 1s - loss: 2.5175 - accuracy: 0.2930 - 1s/epoch - 3ms/step
Epoch 4/60
329/329 - 1s - loss: 2.3806 - accuracy: 0.3311 - 1s/epoch - 3ms/step
Epoch 5/60
329/329 - 1s - loss: 2.2753 - accuracy: 0.3553 - 1s/epoch - 3ms/step
Epoch 6/60
329/329 - 1s - loss: 2.1787 - accuracy: 0.3799 - 1s/epoch - 4ms/step
Epoch 7/60
329/329 - 1s - loss: 2.0824 - accuracy: 0.4026 - 1s/epoch - 3ms/step
Epoch 8/60
329/329 - 1s - loss: 1.9930 - accuracy: 0.4375 - 1s/epoch - 3ms/step
Epoch 9/60
329/329 - 1s - loss: 1.9056 - accuracy: 0.4573 - 1s/epoch - 3ms/step
Epoch 10/60
329/329 - 1s - loss: 1.8160 - accuracy: 0.4851 - 1s/epoch - 3ms/step
Epoch 11/60
329/329 - 1s - loss: 1.7391 - accuracy: 0.5047 - 1s/epoch - 3ms/step
Epoch 12/60
329/329 - 1s - loss: 1.6543 - accuracy: 0.5279 - 1s/epoch - 3ms/step
Epoch 13/60
329/329 - 1s - loss: 1.5

In [7]:
# Функция генерации текста
def generate_text(model, tokenizer, input_text, length):
    text_generated = input_text  # начинаем с исходной фразы
    for _ in range(length):
        encoded_input = tokenizer.texts_to_sequences([text_generated[-sequence_length:]])[0] #Получение последних sequence_length символов из text_generated, 
        #их кодирование в числовые токены с помощью токенизатора.
        encoded_input = pad_sequences([encoded_input], maxlen=sequence_length, truncating='pre') #Получение последних sequence_length символов из text_generated, 
        #их кодирование в числовые токены с помощью токенизатора.
        prediction = model.predict(encoded_input, verbose=0) #Получение предсказания от модели для закодированного ввода.
        predicted_char_index = np.argmax(prediction) #Определение индекса символа с самой высокой вероятностью в предсказании модели.
        predicted_char = tokenizer.index_word[predicted_char_index] #Поиск предсказанного символа по его индексу с помощью обратного индекса токенизатора.
        text_generated += predicted_char
    return text_generated

# Оценка перплексии
def calculate_perplexity(model, text, tokenizer, sequence_length):
    encoded_text = tokenizer.texts_to_sequences([text])[0] #Подготовка последовательностей из входного текста аналогично предыдущим шагам подготовки данных.
    sequences = []
    for i in range(sequence_length, len(encoded_text)):
        sequences.append(encoded_text[i - sequence_length:i + 1])
    sequences = np.array(sequences)
    X, y = sequences[:,:-1], sequences[:,-1]
    X = pad_sequences(X, maxlen=sequence_length)
    y = to_categorical(y, num_classes=vocab_size)
    
    # Вычисление кросс-энтропии
    predictions = model.predict(X) #Получение предсказаний от модели для всех входных последовательностей.
    cross_entropy = categorical_crossentropy(y, predictions) #Вычисление кросс-энтропии между истинными значениями y и предсказаниями predictions
    perplexity = exp(mean(cross_entropy)) #Расчёт перплексии как экспоненциала средней кросс-энтропии по всем примерам в наборе данных.
    return perplexity.numpy()

In [8]:
# Генерация текста
starting_text = "старик"
generated_text = generate_text(model, tokenizer, starting_text, 200)

# Вычисление перплексии
perplexity = calculate_perplexity(model, text, tokenizer, sequence_length)

print(generated_text)
print(f"Перплексия: {perplexity}")


329/329 [==============================] - 1s 2ms/step
старика взаше? з рыня рыбка!
чтом то стеей будетно, моние моне пока,шего грастоегому старина изба тво ведя рюбядею призин е мет, нак стриматвоводик пыще не рогза, не дабыбозей досидая сунные призались;невел
Перплексия: 1.2859749794006348
